# LT5-LE7 Normalized TOA Reflectance and TC Composites

Here, I use radiometric normalization (after Hall et al. 1991) to derive a consistent tasseled cap (TC) time series from Landsat 5 TM and Landsat 7 ETM+ top-of-atmosphere (TOA) data. The reference image is chosen from a Landsat 7 ETM+ annual composite series, given the superior radiometric calibration of ETM+ and the need for as many potential rectification/ normalization target pixels as possible (hence, composites). The ETM+ composite with the highest dynamic range is chosen as the reference image. As described by Liu et al. (2016, p.354), this procedure, though applied to TOA images, should also mitigate variation between images due to the changing atmosphere:

> "A relative radiometric normalisation method using spectrally pseudo-invariant features (PIFs) was used to reduce atmospheric and other unexpected variations between the paired images by adjusting the radiometric properties of the Landsat 5 TM images to match the Landsat 7 ETM+ images with superior radiometric calibration."

Though Liu et al. (2016) used a different normalization approach, Hall et al.'s (1991) method is the best-documented and is conceptually similar to later approaches.

Normalization targets, ideally pseudo-invariant features (PIFs), are chosen to be bright (or dark) pixels with low greenness, based on the tasseled cap/ Kauth-Thomas (KT) transformation. To automate the selection of these PIFs, an image of the sum of the variance across the time series is also used to screen for low-variance, high brightness (or darkness), and low greenness pixels. For the subject images, normalization is applied to individual images, which are later composited.

> Hall, F. G., D. E. Strebel, J. E. Nickeson, and S. J. Goetz. 1991. Radiometric rectification: Toward a common radiometric response among multidate, multisensor images. Remote Sensing of Environment 35 (1):11–27.

> Liu, Q., G. Liu, C. Huang, C. Xie, L. Chu, and L. Shi. 2016. Comparison of tasselled cap components of images from Landsat 5 Thematic Mapper and Landsat 7 Enhanced Thematic Mapper Plus. Journal of Spatial Science 61 (2):351–365.

In [1]:
import ee
from IPython.display import Image
# Example iamge display:
# image = ee.Image('srtm90_v4')
# Image(url=image.getThumbUrl({'min':0, 'max': 3000}))

ee.Initialize()

# Detroit MSA bounds
detroit_msa = ee.FeatureCollection('ft:1QZFRMLeORVfc9sYNOOKD2WyOlEyI8gmj-g3CdTlH', 'geometry')
detroit_metro = detroit_msa.filter(ee.Filter.inList('name', 
    ee.List(['Wayne', 'Oakland', 'Macomb']))); # Smaller area for sampling

# Water mask
water = ee.Image(1).where(ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('max_extent'), ee.Image(0))

# No other way, apparently, to get a range as a list; can't map type coercion client-side...
lt5_dates = ee.List([
    '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
    '2007', '2008', '2009', '2010', '2011'
])

le7_dates = ee.List([
    '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
    '2013', '2014', '2015'
])

combined_dates = ee.List([
    '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
    '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'
])

# A common filter for selecting Landsat images
common_filter = ee.Filter([
    ee.Filter.date('1995-01-01', '2015-12-31'),
    ee.Filter.dayOfYear(121, 273),
    ee.Filter.lessThan('CLOUD_COVER_LAND', 40), # For ease of finding rectification targets
    ee.Filter.eq('WRS_PATH', 20), # NOTE: Enforcing a single path (single view angle per pixel)
])

## Functions

In [2]:
def annual_composite(collection, year):
    '''
    Creates an annual composite for a given year; filters to the year 
    specified, applies the cloud mask, reduces to the median value, 
    and renames the bands..
    '''
    return ee.Image(collection.filterDate(
        ee.String(year).cat(ee.String('-01-01')), 
        ee.String(year).cat(ee.String('-12-31')))\
            .map(cloud_mask)\
            .median()\
            .rename(['blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])\
            .select('blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2')\
            .set({'year': year}))


def cloud_mask(img):
    '''
    Applies a cloud mask to an image based on the QA band of the
    Landsat data.
    '''
    # Bits 3 and 5 are cloud shadow and cloud, respectively
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()
    water_mask = ee.Number(2).pow(2).int() # Bit 2 is water

    # Get the pixel QA band
    qa = img.select('pixel_qa')

    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0)\
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))\
        .And(qa.bitwiseAnd(water_mask).eq(0))\
        .reduceNeighborhood(ee.Reducer.anyNonZero(), ee.Kernel.square(1))
    # NOTE: Also mask those pixels that are adjacent to cloud, cloud shadow, or water

    # Return the masked image, scaled to [0, 1]
    return img.updateMask(mask).divide(10000)\
        .select(
            ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'],
            ['blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])\
        .set({
            'date': img.get('SENSING_TIME'),
            'year': ee.String(img.get('SENSING_TIME')).slice(0, 4)
        })
        
        
def dynamic_range(img, last):
    '''
    Calculates the dynamic range of an image; used as an iterator.
    '''
    names = img.bandNames()
    dr = img.clamp(0.0, 16000.0)\
        .reduceRegion(geometry = detroit_msa, reducer = ee.Reducer.minMax(),
            scale = 30, maxPixels = 95e6)
  
    # Iterate over the list of band names, selecting
    #   the max and min values computed for each
    return ee.List(last).add(
        # Calculate dynamic range as max() - min()
        ee.List(names.iterate(
            lambda b, prev: ee.List(prev).add(
                ee.Number(dr.get(ee.String(b).cat('_').cat('max')))\
                    .subtract(ee.Number(dr.get(ee.String(b).cat('_').cat('min')))))), 
                ee.List([])))\
        .reduce(ee.Reducer.mean()) # Finally, calculate the mean across bands
        

def rescale(img):
    '''
    Converts reflectance to Landsat reflectance units: [0, 16000].
    '''
    return ee.Image(img.multiply(10e3).copyProperties(img))
        
        
def rndsi(img):
    '''
    Calculates the ratio normalized difference soil index (RNDSI).
    '''
    region = detroit_metro.union().geometry()
    ndsi = img.normalizedDifference(['SWIR2', 'green'])

    # Get the band statistics for NDSI, Tasseled Cap
    ndsi_stats = ndsi.reduceRegion(ee.call('Reducer.minMax'), region, 30)
    tc_stats = img.select('B', 'G', 'W')\
        .reduceRegion(ee.call('Reducer.minMax'), region, 30, None, None, False, 24000000)

    # Normalized NDSI and TCB
    nndsi = ndsi.subtract(ee.Image.constant(ndsi_stats.get('nd_min')))\
        .divide(ee.Image.constant(ee.Number(ndsi_stats.get('nd_max'))\
              .subtract(ee.Number(ndsi_stats.get('nd_min')))))
    ntcb = img.select('B').subtract(ee.Image.constant(tc_stats.get('B_min')))\
        .divide(ee.Image.constant(ee.Number(tc_stats.get('B_max'))\
              .subtract(ee.Number(tc_stats.get('B_min')))))

    return nndsi.divide(ntcb).set({'year': img.get('year'), 'variable': 'RNDSI'})


def tasseled_cap(img, b_coefs, g_coefs, w_coefs):
    '''
    Creates a Tasseled Cap-transformed image using the provided brightness,
    greenness, and wetness coefficients.
    '''
    summ = ee.call('Reducer.sum')
    brightness = img.multiply(b_coefs).reduce(summ)
    greenness = img.multiply(g_coefs).reduce(summ)
    wetness = img.multiply(w_coefs).reduce(summ)
    return brightness\
        .addBands(greenness)\
        .addBands(wetness)\
        .select([0,1,2], ['B','G','W'])
        
        
def tasseled_cap_etm_plus(img):
    '''
    Create a tasseled-cap transformation of bands using Landsat top-of-atmosphere 
    (TOA) reflectance coefficients. From Table 2:
        Liu, Q., G. Liu, C. Huang, C. Xie, L. Chu, and L. Shi. 2016. 
        Comparison of tasselled cap components of images from Landsat 5 
        Thematic Mapper and Landsat 7 Enhanced Thematic Mapper Plus. 
        Journal of Spatial Science 61 (2):351–365.
    '''
    img2 = ee.Image(img).select(['blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2'])
    b_coefs = ee.Image([ 0.3561, 0.3972, 0.3904, 0.6966, 0.2286, 0.1596])
    g_coefs = ee.Image([-0.3344,-0.3544,-0.4556, 0.6966,-0.0242,-0.2630])
    w_coefs = ee.Image([ 0.2626, 0.2141, 0.0926, 0.0656,-0.7629,-0.5388])
    return tasseled_cap(img2, b_coefs, g_coefs, w_coefs)\
        .set({
            'year': ee.Image(img).get('year'),
            'system:time_start': ee.Image(img).get('system:time_start')
        })
        
        
def tasseled_cap_tm(img):
    '''
    Create a tasseled-cap transformation of bands using Landsat 
    surface-reflectance coefficients. From Table 1:
        Crist, E. (1985). A TM Tasseled Cap Equivalent
        Transformation for Reflectance Factor Data.
        Remote Sensing of Environment 17: 301-306
    '''
    img2 = ee.Image(img).select(['blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2'])
    b_coefs = ee.Image([ 0.2043, 0.4158, 0.5524, 0.5741, 0.3124, 0.2303])
    g_coefs = ee.Image([-0.1603,-0.2819,-0.4934, 0.7940,-0.0002,-0.1446])
    w_coefs = ee.Image([ 0.0315, 0.2021, 0.3102, 0.1594,-0.6806,-0.6109])
    return tasseled_cap(img2, b_coefs, g_coefs, w_coefs)\
        .set({
            'year': ee.Image(img).get('year'),
            'system:time_start': ee.Image(img).get('system:time_start')
        })


def toa_annual_composite(collection, year):
    '''
    Creates an annual composite for a given year, for input TOA data, 
    applies the cloud mask, reduces to the median value, and renames the bands.
    '''
    return ee.Image(collection.filterDate(
        ee.String(year).cat(ee.String('-01-01')), 
        ee.String(year).cat(ee.String('-12-31')))\
            .map(toa_cloud_mask)\
            .median()\
            .select('blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2')\
            .set({'year': year}))


def toa_cloud_mask(img):
    '''
    Applies a cloud mask to TOA data based on the "BQA" or quality
    assessment band.
    '''
    # Bits 3 and 5 are cloud shadow and cloud, respectively
    cloud_shadow_bit_mask = ee.Number(2).pow(8).int()
    clouds_bit_mask = ee.Number(2).pow(6).int()
    fill_bit_mask = ee.Number(2).pow(0).int() # Bit 2 is water
    snow_ice_bit_mask = ee.Number(2).pow(10).int()
    cirrus_bit_mask = ee.Number(2).pow(12).int()

    # Get the pixel QA band
    qa = img.select('BQA')

    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0)\
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))\
        .And(qa.bitwiseAnd(fill_bit_mask).eq(0))\
        .And(qa.bitwiseAnd(snow_ice_bit_mask).eq(0))\
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    # Return the masked image
    return img.updateMask(mask)\
        .select('blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2')

### Rectification/ Normalization Functions

In [3]:
def get_hall_rectification(tc_func):
    '''
    Returns a function that performs relative radiometric normalization.
    Generates rectification subject means for bright and dark targets, calculates 
    the linear transformation coefficients, then applies the rectification using 
    Hall et al.'s (1991) method. This is a longer version of 
    rectification_reference_means(), which is called for the reference image.
    '''
    def main(img):
        band_names = ee.Image(img).bandNames()
        img_tc = tc_func(img)
        img_bright = img_tc.select('B')
        img_green = img_tc.select('G')

        # Get percentiles of brightness, greenness
        pctb = img_bright.reduceRegion(geometry = detroit_msa, 
           reducer = ee.Reducer.percentile([1, 95]), scale = 30, maxPixels = 16e6)
        pctg = img_green.reduceRegion(geometry = detroit_msa, 
           reducer = ee.Reducer.percentile([10, 20]), scale = 30, maxPixels = 16e6)

        # NOTE: When print()ing a Dictionary, underscores are replaced with spaces!
        # Ideal rectification targets have low greenness and high (or low) brightness
        dark = ee.Image(img_bright).lt(ee.Image.constant(ee.Number(pctb.get('B_p1'))))
        bright = ee.Image(img_bright).gt(ee.Image.constant(ee.Number(pctb.get('B_p95'))))
        green = ee.Image(img_green).lt(ee.Image.constant(ee.Number(pctg.get('G_p20'))))
        potential_rect_areas = dark.addBands(bright).rename(['dark', 'bright'])

        # Mask the high-variance pixels in the Bright and Dark bands
        mask = potential_rect_areas\
            .multiply(var_threshold)\
            .multiply(green) # Should also have low greenness

        # Use this mask to mask out high-variance and non-Bright or non-Dark pixels
        sr_brights = ee.Image(img).updateMask(mask.select('bright'))
        sr_darks = ee.Image(img).updateMask(mask.select('dark'))

        bright_means = ee.Image(sr_brights).reduceRegion(geometry = detroit_msa, 
           reducer = ee.Reducer.mean(), scale = 30, maxPixels = 95e6)

        dark_means = ee.Image(sr_darks).reduceRegion(geometry = detroit_msa, 
           reducer = ee.Reducer.mean(), scale = 30, maxPixels = 95e6)

        # Create constant value images of the means in each band,
        #   allowing that some raw images have zero rectification targets
        #   in view, which means no mean could be calculated and we
        #   return null instead
        bright_sub = ee.Algorithms.If(bright_means.get('blue'),
            ee.Image(bright_means.toArray(band_names)).arrayFlatten([band_names]), # If True
            None) # If False

        dark_sub = ee.Algorithms.If(dark_means.get('blue'),
            ee.Image(dark_means.toArray(band_names)).arrayFlatten([band_names]),
            None)

        # Calculate the coefficients of the linear transformation, allowing
        #   that some raw images might have zero rectification targets in view,
        #   which means we're returning null
        m = ee.Algorithms.If(bright_sub,
            ee.Algorithms.If(dark_sub,
                bright_ref.subtract(dark_ref).divide(ee.Image(bright_sub).subtract(dark_sub)),
                None),
            None)
        b = ee.Algorithms.If(bright_sub,
            ee.Algorithms.If(dark_sub,
                dark_ref.multiply(bright_sub).subtract(ee.Image(dark_sub).multiply(bright_ref))\
                  .divide(ee.Image(bright_sub).subtract(dark_sub)),
                None),
            None)

        # Return the transformed image
        return ee.Algorithms.If(m, img.multiply(m).add(b), None)
    
    return main


def rectification_reference_means(img, prev):
    '''
    Finds rectification target means for the reference ETM+ plus image;
    used as an iterator function.
    '''
    img_tc = tasseled_cap_etm_plus(img)
    img_bright = img_tc.select('B')
    img_green = img_tc.select('G')

    # Get percentiles of brightness, greenness
    pctb = img_bright.reduceRegion(geometry = detroit_msa, 
       reducer = ee.Reducer.percentile([1, 95]), scale = 30, maxPixels = 16e6)
    pctg = img_green.reduceRegion(geometry = detroit_msa, 
       reducer = ee.Reducer.percentile([10, 20]), scale = 30, maxPixels = 16e6)

    # NOTE: When print()ing a Dictionary, underscores are replaced with spaces!
    dark = ee.Image(img_bright).lt(ee.Image.constant(ee.Number(pctb.get('B_p1'))))
    bright = ee.Image(img_bright).gt(ee.Image.constant(ee.Number(pctb.get('B_p95'))))
    green = ee.Image(img_green).lt(ee.Image.constant(ee.Number(pctg.get('G_p20'))))
    potential_rect_areas = dark.addBands(bright).rename(['dark', 'bright'])

    # Mask the high-variance pixels in the Bright and Dark bands
    mask = potential_rect_areas\
        .multiply(var_threshold)\
        .multiply(green) # Should also have low greenness

    # Use this mask to mask out high-variance and non-Bright or non-Dark pixels
    sr_brights = ee.Image(img).updateMask(mask.select('bright'))
    sr_darks = ee.Image(img).updateMask(mask.select('dark'))

    result_bright = ee.Image(sr_brights).reduceRegion(geometry = detroit_msa, 
       reducer = ee.Reducer.mean(), scale = 30, maxPixels = 95e6)

    result_dark = ee.Image(sr_darks).reduceRegion(geometry = detroit_msa, 
       reducer = ee.Reducer.mean(), scale = 30, maxPixels = 95e6)

    return ee.List(prev).add(ee.List([result_bright, result_dark]))

## Assets

In [4]:
lt5_sr = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
    .filter(common_filter)\
    .filterBounds(detroit_msa)
lt5_toa = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')\
    .filter(common_filter)\
    .filterBounds(detroit_msa)\
    .map(lambda img: img\
        .select('B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'BQA')\
        .rename(['blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2', 'BQA']))

le7_toa = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')\
    .filter(common_filter)\
    .filterBounds(detroit_msa)\
    .map(lambda img: img\
        .select('B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'BQA')\
        .rename(['blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2', 'BQA']))

# Water mask
water = ee.Image(1).where(ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('max_extent'), ee.Image(0))

# Look at 2013, 2013, and 2015 CDL and choose 1 where pixel is always non-cultivated, zero otherwise
# Cultivated = 2 in the "cultivated" band
cdl = ee.ImageCollection('USDA/NASS/CDL')\
    .filterBounds(detroit_msa)\
    .filter(ee.Filter([
        ee.Filter.inList('system:index', ee.List(['2013', '2014', '2015']))
    ]))\
    .map(lambda img: img.select('cultivated').lt(2))\
    .reduce(ee.Reducer.allNonZero())\
    .clip(ee.Feature(detroit_msa.union().toList(1).get(0)).bounds().geometry())

### Landsat Collections

In [5]:
# Individual TOA scenes
lt5_toa_collection = ee.ImageCollection(lt5_toa.map(toa_cloud_mask))
le7_toa_collection = ee.ImageCollection(le7_toa.map(toa_cloud_mask))

### Landsat Composites

In [6]:
# The SR composite is used ONLY for selecting reference rectification targets
lt5_sr_composites = ee.ImageCollection(
    lt5_dates.map(lambda year: ee.Image(annual_composite(lt5_sr\
        .select('B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'), year))))\
    .map(rescale)

# The TOA composites are the "business end"
lt5_toa_composites = ee.ImageCollection(
    lt5_dates.map(lambda year: ee.Image(toa_annual_composite(lt5_toa, year))))\
    .map(rescale)

le7_toa_composites = ee.ImageCollection(
    le7_dates.map(lambda year: ee.Image(toa_annual_composite(le7_toa, year))))\
    .map(rescale)

## Workspace

### Step 1: Calculating Dynamic Range

The intent here is to find the reference image (from among available TOA composites) with the highest dynamic range.

In [35]:
dr_results = le7_toa_composites\
    .map(lambda img: img.updateMask(water))\
    .iterate(dynamic_range, ee.List([]))

print(dr_results)

EEException: Required argument (first) missing to function: Iterate an algorithm over a list.  The algorithm is expected to take two
objects, the current list item, and the result from the previous iteration
or the value of first for the first iteration.

Args:
  list
  function
  first

### Step 2: Calculating Multi-Temporal Variance

In [7]:
var_map = lt5_sr_composites\
    .reduce(ee.Reducer.variance())\
    .reduce(ee.Reducer.sum())
  
# Choose a variance threshold from the histogram
# histogram = ui.Chart.image.histogram(var_map.clamp(0, 1e6), detroit_metro, 30)\
#     .setSeriesNames(['Variance'])
# print(histogram)
    
# Filter to those pixels with variance less than the ~1 percentile
var_threshold = var_map.expression('sum <= 20000', {
  'sum': var_map.select(0)
})

### Step 3: Calculate KT Transform for LT5 and Select Reference Targets

In [8]:
# Get rectification target means for the reference image
# NOTE: Index 6 is the 2007 image, the chosen reference composite
ref_image = ee.ImageCollection(le7_toa_composites).toList(7).get(6)

le7_rect_target_means = ee.List([ref_image])\
    .iterate(rectification_reference_means, ee.List([]))

# Ugh. This incredibly convoluted expression is here because there is easy way to convert
#    a Dictionary to a List from which to create a multi-band constant image.
bright_ref = ee.Image(ee.Dictionary(ee.List(ee.List(le7_rect_target_means).get(0)).get(0))\
        .toArray(ee.Image(le7_toa_composites.first()).bandNames()))\
    .arrayFlatten([ee.Image(le7_toa_composites.first()).bandNames()])

# NOTE: Second element (at index 1) is the dark band  
dark_ref = ee.Image(ee.Dictionary(ee.List(ee.List(le7_rect_target_means).get(0)).get(1))\
        .toArray(ee.Image(le7_toa_composites.first()).bandNames()))\
    .arrayFlatten([ee.Image(le7_toa_composites.first()).bandNames()])

### Step 4: Calculate the Coefficients of the Normalization, Apply, Export

In [9]:
def get_unique_dates(img, prev):
    '''
    Returns a list of unique dates from a collection of images, when
    used as an iterator function.
    '''
    date = img.get('DATE_ACQUIRED')
    # Get a list of *unique* dates only
    return ee.Algorithms.If(ee.List(prev).contains(date),
        ee.List(prev),
        ee.List(prev).add(date))


def normalized_median_composite(year, prev):
    # Need to convert these FeatureCollections, which actually contain Images
    #   (no idea how they ended up that way), into ImageCollections
    lt5_set = ee.Algorithms.If(ee.Algorithms.IsEqual(lt5_toa_normalized.size(), 0),
        ee.ImageCollection([]),
        ee.ImageCollection(lt5_toa_normalized.toList(lt5_toa_normalized.size()))\
            .filterMetadata('year', 'equals', year))
    
    le7_set = ee.Algorithms.If(ee.Algorithms.IsEqual(le7_toa_normalized.size(), 0),
        ee.ImageCollection([]),
        ee.ImageCollection(le7_toa_normalized.toList(le7_toa_normalized.size()))\
            .filterMetadata('year', 'equals', year))

    merged = ee.ImageCollection(ee.ImageCollection(lt5_set)\
        .merge(ee.ImageCollection(le7_set)));
    
    return ee.ImageCollection(prev).merge(ee.ImageCollection([merged\
        .median()\
        .set({
            'year': year,
            'size': ee.Number(ee.ImageCollection(lt5_set).size())\
                .add(ee.ImageCollection(le7_set).size())
        })
    ]))


def normalized_tc_first_max_composite(year, prev):
    # Need to convert these FeatureCollections, which actually contain Images
    #   (no idea how they ended up that way), into ImageCollections
    lt5_set = ee.Algorithms.If(ee.Algorithms.IsEqual(lt5_toa_normalized.size(), 0),
        ee.ImageCollection([]),
        ee.ImageCollection(lt5_toa_normalized.toList(lt5_toa_normalized.size()))\
            .filterMetadata('year', 'equals', year))
    
    le7_set = ee.Algorithms.If(ee.Algorithms.IsEqual(le7_toa_normalized.size(), 0),
        ee.ImageCollection([]),
        ee.ImageCollection(le7_toa_normalized.toList(le7_toa_normalized.size()))\
            .filterMetadata('year', 'equals', year))

    merged = ee.ImageCollection(ee.ImageCollection(lt5_set)\
        .merge(ee.ImageCollection(le7_set)));
    
    # Note we apply the TC transformation here *first,* before compositing
    return ee.ImageCollection(prev).merge(ee.ImageCollection([merged\
        .map(tasseled_cap_etm_plus)\
        .max()\
        .set({
            'year': year,
            'size': ee.Number(ee.ImageCollection(lt5_set).size())\
                .add(ee.ImageCollection(le7_set).size())
        })
    ]))


def rectify_etm_plus(img):
    rectified = get_hall_rectification(tasseled_cap_etm_plus)(img)
    return ee.Algorithms.If(rectified,
        ee.Image(rectified)
            .updateMask(water)
            .set({
                'date': img.get('DATE_ACQUIRED'),
                'year': ee.String(img.get('DATE_ACQUIRED')).slice(0, 4)
            }), None)


def rectify_tm(img):
    rectified = get_hall_rectification(tasseled_cap_tm)(img)
    return ee.Algorithms.If(rectified,
        ee.Image(rectified)
            .updateMask(water)
            .set({
                'date': img.get('DATE_ACQUIRED'),
                'year': ee.String(img.get('DATE_ACQUIRED')).slice(0, 4)
            }), None)

### Normalized Composite: Tasseled Cap of Median Reflectance

In [ ]:
for year in combined_dates:
    year = ee.String(str(year))
    lt5_toa_collection_minor = lt5_toa_collection\
        .filterDate( # Get just images in the given year(s)
            ee.String(year).cat(ee.String('-01-01')), 
            ee.String(year).cat(ee.String('-12-31')))

    le7_toa_collection_minor = le7_toa_collection\
        .filterDate( # Get just images in the given year(s)
            ee.String(year).cat(ee.String('-01-01')), 
            ee.String(year).cat(ee.String('-12-31')))

    # Get a list of the *unique* dates in these year(s)
    lt5_toa_collection_dates = lt5_toa_collection_minor\
        .iterate(get_unique_dates, ee.List([]))

    le7_toa_collection_dates = le7_toa_collection_minor\
        .iterate(get_unique_dates, ee.List([]))

    # Mosaic images from the same date in these year(s)
    lt5_toa_collection_mosaics = ee.List(lt5_toa_collection_dates)\
        .iterate(lambda date, prev: ee.ImageCollection(prev).merge(ee.ImageCollection([
            lt5_toa_collection_minor\
                .filter(ee.Filter.eq('DATE_ACQUIRED', ee.String(date)))\
                .median()\
                .set({'DATE_ACQUIRED': date})
        ])), ee.ImageCollection([]))

    le7_toa_collection_mosaics = ee.List(le7_toa_collection_dates)\
        .iterate(lambda date, prev: ee.ImageCollection(prev).merge(ee.ImageCollection([
            le7_toa_collection_minor\
                .filter(ee.Filter.eq('DATE_ACQUIRED', ee.String(date)))\
                .median()\
                .set({'DATE_ACQUIRED': date})
        ])), ee.ImageCollection([]))

    # Then, rectify each image in those year(s); 
    #   should ignore images without any unmasked rectification targets, e.g., LT05_020030_19980821
    lt5_toa_normalized = ee.ImageCollection(lt5_toa_collection_mosaics)\
        .map(rectify_tm, True); # dropNulls = true
    le7_toa_normalized = ee.ImageCollection(le7_toa_collection_mosaics)\
        .map(rectify_etm_plus, True); # dropNulls = true

    # Finally, merge the LT5 and LE7 collections in each year,
    #   then make a median pixel composite
    final_composites = ee.List([ee.String(year)]).iterate(normalized_median_composite, 
        ee.ImageCollection([]));

    # Pop off the first (N for N years) composite image(s)
    result = ee.Image(ee.ImageCollection(final_composites).toList(1).get(0))\
        .updateMask(cdl) # Finally, apply the CDL as a mask

    # NOTE: The normalized TM data should be transformed with the ETM+ coefficients
    result_tc = tasseled_cap_etm_plus(result)
    result_rndsi = rndsi(ee.Image(result_tc).addBands(result, ['green', 'SWIR2']))
    
    # Get a year for the filename/ description
    the_year = list(range(1995, 2016))[i]
    
    task = ee.batch.Export.image.toDrive(image = result_tc, 
        description = 'LT5-LE7_normalized_TOA_composite_TC_%d' % the_year, 
        region = detroit_msa.geometry().bounds().getInfo()['coordinates'],
        folder = 'EarthEngine', scale = 30, crs = 'EPSG:32617')
    task.start()
    
    task = ee.batch.Export.image.toDrive(image = result_rndsi, 
        description = 'LT5-LE7_normalized_TOA_composite_%d_RNDSI' % the_year, 
        region = detroit_msa.geometry().bounds().getInfo()['coordinates'],
        folder = 'EarthEngine', scale = 30, crs = 'EPSG:32617')
    task.start()

### Normalized Composite: Maximum Value Composite

Here, the TC transformation is applied prior to compositing, where the maximum value in each band is taken. **The goal is to obtain the maximum TC Greenness.**

In [19]:
for i, year in enumerate(xrange(1995, 1996)):
    year = ee.String(str(year))
    lt5_toa_collection_minor = lt5_toa_collection\
        .filterDate( # Get just images in the given year(s)
            ee.String(year).cat(ee.String('-01-01')), 
            ee.String(year).cat(ee.String('-12-31')))

    le7_toa_collection_minor = le7_toa_collection\
        .filterDate( # Get just images in the given year(s)
            ee.String(year).cat(ee.String('-01-01')), 
            ee.String(year).cat(ee.String('-12-31')))

    # Get a list of the *unique* dates in these year(s)
    lt5_toa_collection_dates = lt5_toa_collection_minor\
        .iterate(get_unique_dates, ee.List([]))

    le7_toa_collection_dates = le7_toa_collection_minor\
        .iterate(get_unique_dates, ee.List([]))

    # Mosaic images from the same date in these year(s)
    lt5_toa_collection_mosaics = ee.List(lt5_toa_collection_dates)\
        .iterate(lambda date, prev: ee.ImageCollection(prev).merge(ee.ImageCollection([
            lt5_toa_collection_minor\
                .filter(ee.Filter.eq('DATE_ACQUIRED', ee.String(date)))\
                .median()\
                .set({'DATE_ACQUIRED': date})
        ])), ee.ImageCollection([]))

    le7_toa_collection_mosaics = ee.List(le7_toa_collection_dates)\
        .iterate(lambda date, prev: ee.ImageCollection(prev).merge(ee.ImageCollection([
            le7_toa_collection_minor\
                .filter(ee.Filter.eq('DATE_ACQUIRED', ee.String(date)))\
                .median()\
                .set({'DATE_ACQUIRED': date})
        ])), ee.ImageCollection([]))

    # Then, rectify each image in those year(s); 
    #   should ignore images without any unmasked rectification targets, e.g., LT05_020030_19980821
    lt5_toa_normalized = ee.ImageCollection(lt5_toa_collection_mosaics)\
        .map(rectify_tm, True); # dropNulls = true
    le7_toa_normalized = ee.ImageCollection(le7_toa_collection_mosaics)\
        .map(rectify_etm_plus, True); # dropNulls = true

    # Finally, merge the LT5 and LE7 collections in each year,
    #   then make a median pixel composite
    final_composites = ee.List([ee.String(year)]).iterate(normalized_tc_first_max_composite, 
        ee.ImageCollection([]));

    # Pop off the first (N for N years) composite image(s)
    result = ee.Image(ee.ImageCollection(final_composites).toList(1).get(0))\
        .updateMask(cdl) # Finally, apply the CDL as a mask

    # NOTE: The normalized TM data should be transformed with the ETM+ coefficients
    result_tc = result.select('G')
    
    # Get a year for the filename/ description
    the_year = list(range(1995, 2016))[i]
    
    task = ee.batch.Export.image.toDrive(image = result_tc, 
        description = 'LT5-LE7_normalized_TOA_composite_TC_Greenness_%d' % the_year, 
        region = detroit_msa.geometry().bounds().getInfo()['coordinates'],
        folder = 'EarthEngine', scale = 30, crs = 'EPSG:32617')
    task.start()